In [ ]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(ShinyCell)
    library(dplyr)
    library(SingleR)
    library(dplyr)
})

In [ ]:
counts <- readRDS("data/2023-04-05_rawCounts_Sc_QCed.rds")
metadata <- readRDS("data/2023-04-05_metaData_Sc_QCed.rds")

obj <- CreateSeuratObject(counts = counts, meta.data = metadata, min.cells = 3, min.features = 300)
obj
save(obj, file = "objects/00_raw.rda")

In [ ]:
# QC
obj[["percent.mt"]] <- PercentageFeatureSet(object = obj, pattern = "^MT-")
obj
save(obj, file = file.path("objects", "01.rda"))

In [ ]:
# Normalization
obj <- SCTransform(obj, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi', conserve.memory = TRUE)
save(obj, file=file.path("objects", "02.rda"))

In [ ]:
load(file=file.path("objects", "02.rda"))

In [ ]:
source("R/process.R")

In [22]:
### Write shiny files

#bcell
seu =  get(load(file = "objects/BCellmonaco05.rda"))
colnames(seu@meta.data)[10] <- "Monaco"
scConf1 = createConfig(seu)
makeShinyFiles(seu, scConf1,
             gene.mapping = TRUE,
             shiny.prefix = "sc1",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

#myeloid
seu =  get(load(file = "objects/MyeloidCellmonaco05.rda"))
colnames(seu@meta.data)[10] <- "Monaco"
scConf2 = createConfig(seu)
makeShinyFiles(seu, scConf2,
             gene.mapping = TRUE,
             shiny.prefix = "sc2",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

#nk
seu =  get(load(file = "objects/NKCellmonaco05.rda"))
colnames(seu@meta.data)[10] <- "Monaco"
scConf3 = createConfig(seu)
makeShinyFiles(seu, scConf3,
             gene.mapping = TRUE,
             shiny.prefix = "sc3",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

#plasma
seu =  get(load(file = "objects/PlasmaCellmonaco05.rda"))
colnames(seu@meta.data)[10] <- "Monaco"
scConf4 = createConfig(seu)
makeShinyFiles(seu, scConf4,
             gene.mapping = TRUE,
             shiny.prefix = "sc4",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

#tcell
seu =  get(load(file = "objects/TCellmonaco05.rda"))
colnames(seu@meta.data)[10] <- "Monaco"
scConf5 = createConfig(seu)
makeShinyFiles(seu, scConf5,
             gene.mapping = TRUE,
             shiny.prefix = "sc5",
             shiny.dir = "amp2_sle_scRNAseq_immune/",
             gex.assay = "SCT")

makeShinyCodesMulti(
  shiny.title = "AMP Phase II SLE, Immune Cells", shiny.footnotes = NULL,
  shiny.prefix = c("sc1", "sc2", "sc3", "sc4", "sc5"),
  shiny.headers = c("B Cell", "Myeloid", "NK", "Plasma Cell", "T Cell"), 
  shiny.dir = "amp2_sle_scRNAseq_immune/") 

ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
nFeature_RNA,nFeature_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Control|LN,#A6CEE3|#B15928,1,TRUE,FALSE
Tissue_Type,Tissue_Type,Immune,black,1,FALSE,FALSE
Fine_Cell_Type,Fine_Cell_Type,0. CD16+ CX3CR1+ Mono|1. CD14+ CD16+ Mono|10. SELENOP+ LYVE1lo Mac|11. C1Qhi GPNMBhi Mac|12. SELENOP+ LYVE1hi Mac|13. CCR7+ LAMP3+ cDC2|14. APOC1hi C3hi Mac|15. MHC2lo cDC2|16. APOC1lo C3hi Mac|17. MHC2hi cDC2|18. CENPF+ MKI67+ Proliferating|19. cDC1|2. CD14+ CCR2+ Mono|20. pDC|3. CD14+ CCL2+ Mono|4. Mast Cell|5. C1Qlo NUPR1hi Mac|6. SELENOP+ TIMP1+ Mac|7. C1Qlo FABP5hi SPP1hi Mac|8. SELENOP- TIMP1+ Mac|9. C1Qhi FABP5hi SPP1hi Mac,#A6CEE3|#5B9EC9|#2D82AF|#7EBA98|#98D277|#52AF43|#6F9E4C|#DD9A88|#F16667|#E42022|#F06C45|#FDBB69|#FE982C|#F78620|#D9A295|#B294C7|#7D54A5|#9E8099|#F0EB99|#DBB466|#B15928,6,TRUE,FALSE
Broad_Cell_Type,Broad_Cell_Type,Myeloid Cell,black,1,FALSE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14,#A6CEE3|#3B8ABE|#72B29C|#84C868|#4F9F3B|#EC9A91|#E93E3F|#F06C45|#FDAC4F|#FB820F|#D1AAB7|#8C66AF|#A99099|#EEDB80|#B15928,4,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14,#A6CEE3|#3B8ABE|#72B29C|#84C868|#4F9F3B|#EC9A91|#E93E3F|#F06C45|#FDAC4F|#FB820F|#D1AAB7|#8C66AF|#A99099|#EEDB80|#B15928,4,TRUE,FALSE


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
nFeature_RNA,nFeature_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Control|LN,#A6CEE3|#B15928,1,TRUE,FALSE
Tissue_Type,Tissue_Type,Immune,black,1,FALSE,FALSE
Fine_Cell_Type,Fine_Cell_Type,0.CD56lo NK|3. CD56bright NK,#A6CEE3|#B15928,1,TRUE,FALSE
Broad_Cell_Type,Broad_Cell_Type,NK Cell,black,1,FALSE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6,#A6CEE3|#99CD91|#B89B74|#F06C45|#ED8F47|#825D99|#B15928,2,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6,#A6CEE3|#99CD91|#B89B74|#F06C45|#ED8F47|#825D99|#B15928,2,TRUE,FALSE


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
nFeature_RNA,nFeature_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Control|LN,#A6CEE3|#B15928,1,TRUE,FALSE
Tissue_Type,Tissue_Type,Immune,black,1,FALSE,FALSE
Fine_Cell_Type,Fine_Cell_Type,10. JCHAIN+ IGA+ IGL+ Plasma|11. IGHG+ IGL+ Plasma|6. CD38++ CD27++ IGHD- Plasma|7. IGHG+ IGL- Plasma|8. JCHAIN+ IGA+ IGL- Plasma|9. IGHG+ IGL- Plasma,#A6CEE3|#98D277|#F16667|#FE982C|#7D54A5|#B15928,2,TRUE,FALSE
Broad_Cell_Type,Broad_Cell_Type,Plasma Cell,black,1,FALSE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22,#A6CEE3|#62A3CB|#1F78B4|#68AB9F|#B2DF8A|#72BF5A|#33A02C|#969D62|#FB9A99|#EF595A|#E31A1C|#F06C45|#FDBF6F|#FE9E37|#FF7F00|#E4986B|#CAB2D6|#9A77B8|#6A3D9A|#B49D99|#FFFF99|#D8AC60|#B15928,6,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22,#A6CEE3|#62A3CB|#1F78B4|#68AB9F|#B2DF8A|#72BF5A|#33A02C|#969D62|#FB9A99|#EF595A|#E31A1C|#F06C45|#FDBF6F|#FE9E37|#FF7F00|#E4986B|#CAB2D6|#9A77B8|#6A3D9A|#B49D99|#FFFF99|#D8AC60|#B15928,6,TRUE,FALSE


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
nFeature_RNA,nFeature_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Control|LN,#A6CEE3|#B15928,1,TRUE,FALSE
Tissue_Type,Tissue_Type,Immune,black,1,FALSE,FALSE
Fine_Cell_Type,Fine_Cell_Type,1. CTL|10. GZMK+ CD8+ ITGAE+|11. CD4+ CD28+ CCR7- SELL-|12. CD4?|13. GZMK+ CD8+ CD69+|14. CD4+ CD69+ ITGAE+ S1P1- JUNhi|15. CD4+ CD69+ ITGAE+ S1P1- JUNlo|16. CD4+ CD28+ SELL+ CCR7+|17. ILCs|18. CD4+ RORC+ KLRB1+ CCR6+|19. CD4+ CCR7+ CD28+ SELL+ Ribohi|2. SYNEhi CTL|20. CD4+ IL2RA++ FOXP3++|21. CD4+ SELL+ CD28+ CCR7+ IL2RA+ FOXP3+|22. CD4+ PDCD1+ CXCR5+|4. Proliferating|5. GZMK+ CD8+ NKG7lo|6. GZMK+ CD8 NKG7hi|7. GZMK+ CD8+ CD28+ CCR7- SELL-|8. GZMK+ CD8+ NEAT1+|9. GZMK+ CD8+ CD69+ ITGAE+ S1P1-,#A6CEE3|#5B9EC9|#2D82AF|#7EBA98|#98D277|#52AF43|#6F9E4C|#DD9A88|#F16667|#E42022|#F06C45|#FDBB69|#FE982C|#F78620|#D9A295|#B294C7|#7D54A5|#9E8099|#F0EB99|#DBB466|#B15928,6,TRUE,FALSE
Broad_Cell_Type,Broad_Cell_Type,T Cell,black,1,FALSE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11,#A6CEE3|#1F78B4|#B2DF8A|#33A02C|#FB9A99|#E31A1C|#FDBF6F|#FF7F00|#CAB2D6|#6A3D9A|#FFFF99|#B15928,3,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11,#A6CEE3|#1F78B4|#B2DF8A|#33A02C|#FB9A99|#E31A1C|#FDBF6F|#FF7F00|#CAB2D6|#6A3D9A|#FFFF99|#B15928,3,TRUE,FALSE


In [23]:
system("R -e \"shiny::runApp('amp2_sle_scRNAseq_immune')\"")

In [26]:
### Write shiny files

#dn
seu =  get(load(file = "objects/DNblueprint05.rda"))
colnames(seu@meta.data)[10] <- "Blueprint"
scConf1 = createConfig(seu)
makeShinyFiles(seu, scConf1,
             gene.mapping = TRUE,
             shiny.prefix = "sc1",
             shiny.dir = "amp2_sle_scRNAseq_kidney/",
             gex.assay = "SCT")

#glom
seu =  get(load(file = "objects/GLOMblueprint05.rda"))
colnames(seu@meta.data)[10] <- "Blueprint"
scConf2 = createConfig(seu)
makeShinyFiles(seu, scConf2,
             gene.mapping = TRUE,
             shiny.prefix = "sc2",
             shiny.dir = "amp2_sle_scRNAseq_kidney/",
             gex.assay = "SCT")

#intl
seu =  get(load(file = "objects/INTLblueprint05.rda"))
colnames(seu@meta.data)[10] <- "Blueprint"
scConf3 = createConfig(seu)
makeShinyFiles(seu, scConf3,
             gene.mapping = TRUE,
             shiny.prefix = "sc3",
             shiny.dir = "amp2_sle_scRNAseq_kidney/",
             gex.assay = "SCT")

#loh
seu =  get(load(file = "objects/LOHblueprint05.rda"))
colnames(seu@meta.data)[10] <- "Blueprint"
scConf4 = createConfig(seu)
makeShinyFiles(seu, scConf4,
             gene.mapping = TRUE,
             shiny.prefix = "sc4",
             shiny.dir = "amp2_sle_scRNAseq_kidney/",
             gex.assay = "SCT")

#und   
seu =  get(load(file = "objects/UNDblueprint05.rda"))
colnames(seu@meta.data)[10] <- "Blueprint"
scConf5 = createConfig(seu)
makeShinyFiles(seu, scConf5,
             gene.mapping = TRUE,
             shiny.prefix = "sc11",
             shiny.dir = "amp2_sle_scRNAseq_kidney/",
             gex.assay = "SCT")

#pt
seu =  get(load(file = "objects/PTblueprint05.rda"))
colnames(seu@meta.data)[10] <- "Blueprint"
scConf6 = createConfig(seu)
makeShinyFiles(seu, scConf6,
             gene.mapping = TRUE,
             shiny.prefix = "sc6",
             shiny.dir = "amp2_sle_scRNAseq_kidney/",
             gex.assay = "SCT")

makeShinyCodesMulti(
  shiny.title = "AMP Phase II SLE, Structural Cells", shiny.footnotes = NULL,
  shiny.prefix = c("sc1", "sc2", "sc3", "sc4", "sc5", "sc6"),
  shiny.headers = c("DN", "GLOM", "INTL", "LOH", "UND", "PT"), 
  shiny.dir = "amp2_sle_scRNAseq_kidney/") 

ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
nFeature_RNA,nFeature_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Control|LN,#A6CEE3|#B15928,1,TRUE,FALSE
Tissue_Type,Tissue_Type,Kidney,black,1,FALSE,FALSE
Fine_Cell_Type,Fine_Cell_Type,DCT|DCT/CNT|IC-A 1|IC-A 2|IC-B|MD|PC|PC/CNT,#A6CEE3|#72B29C|#4F9F3B|#E93E3F|#FDAC4F|#D1AAB7|#A99099|#B15928,2,TRUE,FALSE
Broad_Cell_Type,Broad_Cell_Type,DN,black,1,FALSE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18,#A6CEE3|#5399C6|#3F8EAA|#99CD91|#79C360|#3E9F32|#B89B74|#F47676|#E52829|#F06C45|#FDB762|#FE901E|#ED8F47|#CCAFCA|#9471B4|#825D99|#DDD399|#E0BE6D|#B15928,5,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18,#A6CEE3|#5399C6|#3F8EAA|#99CD91|#79C360|#3E9F32|#B89B74|#F47676|#E52829|#F06C45|#FDB762|#FE901E|#ED8F47|#CCAFCA|#9471B4|#825D99|#DDD399|#E0BE6D|#B15928,5,TRUE,FALSE


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
nFeature_RNA,nFeature_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Control|LN,#A6CEE3|#B15928,1,TRUE,FALSE
Tissue_Type,Tissue_Type,Kidney,black,1,FALSE,FALSE
Fine_Cell_Type,Fine_Cell_Type,POD/MC/PEC,black,1,FALSE,FALSE
Broad_Cell_Type,Broad_Cell_Type,GLOM,black,1,FALSE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14,#A6CEE3|#3B8ABE|#72B29C|#84C868|#4F9F3B|#EC9A91|#E93E3F|#F06C45|#FDAC4F|#FB820F|#D1AAB7|#8C66AF|#A99099|#EEDB80|#B15928,4,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14,#A6CEE3|#3B8ABE|#72B29C|#84C868|#4F9F3B|#EC9A91|#E93E3F|#F06C45|#FDAC4F|#FB820F|#D1AAB7|#8C66AF|#A99099|#EEDB80|#B15928,4,TRUE,FALSE


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
nFeature_RNA,nFeature_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Control|LN,#A6CEE3|#B15928,1,TRUE,FALSE
Tissue_Type,Tissue_Type,Kidney,black,1,FALSE,FALSE
Fine_Cell_Type,Fine_Cell_Type,EC 1|EC 2|FIB/VSMC,#A6CEE3|#F06C45|#B15928,1,TRUE,FALSE
Broad_Cell_Type,Broad_Cell_Type,INTL,black,1,FALSE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15,#A6CEE3|#438EC0|#63A8A0|#98D277|#3BA432|#B89B74|#F16667|#E62F27|#F9A963|#FE982C|#ED8F47|#C3AAD2|#7D54A5|#B9A499|#EAD27A|#B15928,4,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15,#A6CEE3|#438EC0|#63A8A0|#98D277|#3BA432|#B89B74|#F16667|#E62F27|#F9A963|#FE982C|#ED8F47|#C3AAD2|#7D54A5|#B9A499|#EAD27A|#B15928,4,TRUE,FALSE


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
nFeature_RNA,nFeature_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Control|LN,#A6CEE3|#B15928,1,TRUE,FALSE
Tissue_Type,Tissue_Type,Kidney,black,1,FALSE,FALSE
Fine_Cell_Type,Fine_Cell_Type,ATL|C-TAL 1|C-TAL 2|C-TAL 3|injuredTAL 1|injuredTAL 2,#A6CEE3|#98D277|#F16667|#FE982C|#7D54A5|#B15928,2,TRUE,FALSE
Broad_Cell_Type,Broad_Cell_Type,LOH,black,1,FALSE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18,#A6CEE3|#5399C6|#3F8EAA|#99CD91|#79C360|#3E9F32|#B89B74|#F47676|#E52829|#F06C45|#FDB762|#FE901E|#ED8F47|#CCAFCA|#9471B4|#825D99|#DDD399|#E0BE6D|#B15928,5,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18,#A6CEE3|#5399C6|#3F8EAA|#99CD91|#79C360|#3E9F32|#B89B74|#F47676|#E52829|#F06C45|#FDB762|#FE901E|#ED8F47|#CCAFCA|#9471B4|#825D99|#DDD399|#E0BE6D|#B15928,5,TRUE,FALSE


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
nFeature_RNA,nFeature_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,LN,black,1,FALSE,FALSE
Tissue_Type,Tissue_Type,Kidney,black,1,FALSE,FALSE
Fine_Cell_Type,Fine_Cell_Type,Undefined 1|Undefined 2,#A6CEE3|#B15928,1,TRUE,FALSE
Broad_Cell_Type,Broad_Cell_Type,UND,black,1,FALSE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2,#A6CEE3|#F06C45|#B15928,1,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2,#A6CEE3|#F06C45|#B15928,1,TRUE,FALSE


ID,UI,fID,fCL,fRow,grp,dimred
<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
orig.ident,orig.ident,SeuratProject,black,1,FALSE,FALSE
nCount_RNA,nCount_RNA,NA,NA,NA,FALSE,FALSE
nFeature_RNA,nFeature_RNA,NA,NA,NA,FALSE,FALSE
Stim,Stim,Control|LN,#A6CEE3|#B15928,1,TRUE,FALSE
Tissue_Type,Tissue_Type,Kidney,black,1,FALSE,FALSE
Fine_Cell_Type,Fine_Cell_Type,injuredPT/DTL|PTS1/S2 1|PTS1/S2 2|PTS1/S2 3|PTS1/S2 4|PTS2/S3 1|PTS2/S3 2,#A6CEE3|#99CD91|#B89B74|#F06C45|#ED8F47|#825D99|#B15928,2,TRUE,FALSE
Broad_Cell_Type,Broad_Cell_Type,PT,black,1,FALSE,FALSE
SCT_snn_res.0.5,SCT_snn_res.0.5,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16,#A6CEE3|#4992C2|#569EA4|#AADB84|#52AF43|#8A9D5B|#F88A89|#E73233|#F06C45|#FDB35A|#FE870D|#E19B78|#B294C7|#70449D|#C7B699|#E6CB75|#B15928,5,TRUE,FALSE
seurat_clusters,seurat_clusters,0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16,#A6CEE3|#4992C2|#569EA4|#AADB84|#52AF43|#8A9D5B|#F88A89|#E73233|#F06C45|#FDB35A|#FE870D|#E19B78|#B294C7|#70449D|#C7B699|#E6CB75|#B15928,5,TRUE,FALSE


In [27]:
system("R -e \"shiny::runApp('amp2_sle_scRNAseq_kidney')\"")